# Lemma dilemma

*Creator: Magnus Struckmann*
## Table of contents

- Notebook summary
- Create data sets

### Notebook summary

Based on the program developed in the lecture (SparseRandomProjections), analyze a database of your choice (but not exactly the same digits data as in the lecture) using random projections.
Study the accuracy (or a score of your choice that makes most sense for your data) as a function of the number of dimensions / features that survived the random projection.

In [ ]:
# Import packages
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from sklearn import datasets

### Create data sets

In [ ]:
data_collection = []


3D scatter plot 
x = dimensions/eps 
y = compute time 
z = accuracy/f1 score
3D heat map 
x = pairwise squared distances in original space 
y = pairwise squared distances in projected space 
z = n_components
3D "histogram"
x = squared distance rates original/projected
y = n_components
z = distribution of sample pairs
#scatter plot accuracy vs compute time (x=)

The Olivetti faces dataset
Classes 40
Samples total 400
Dimensionality 4096
Features real, between 0 and 1

The Labeled Faces in the Wild face recognition dataset
Classes 5749
Samples 13233
Dimensionality 5828
Features real, between 0 and 255

Forest covertypes
Classes 7
Samples 581012
Dimensionality 54
Features int